# R1CS with Integers

In [1]:
import numpy as np

# 1, out, x, y, v1, v2, v3
L = np.array([
    [0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, -5, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 1],
])

R = np.array([
    [0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0],
])

O = np.array([
    [0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 1],
    [0, 1, 0, 0, 0, -1, 0],
])

x = 4
y = -2
v1 = x * x
v2 = v1 * v1         # x^4
v3 = -5*y * y
out = v3*v1 + v2    # -5y^2 * x^2

witness = np.array([1, out, x, y, v1, v2, v3])

assert all(np.equal(np.matmul(L, witness) * np.matmul(R, witness), np.matmul(O, witness))), "not equal"

result = O.dot(witness) == np.multiply(L.dot(witness), R.dot(witness))
assert result.all(), "result contains inequality"

# R1CS in Finite Field aka Galois Field using Galois library

In [2]:
# building intuition
import galois

GF = galois.GF(79)

a = GF(70)
b = GF(10)

print(a + b)
# prints 1

1


In [3]:
# change the matrix values for finite field arithmetic
L = np.array([
    [0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 74, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 1],
])

R = np.array([
    [0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0],
])

O = np.array([
    [0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 1],
    [0, 1, 0, 0, 0, 78, 0],
])

L_galois = GF(L)
R_galois = GF(R)
O_galois = GF(O)

x = GF(4)
y = GF(79-2)
v1 = x * x
v2 = v1 * v1         # x^4
v3 = GF(79-5)*y * y
out = v3*v1 + v2    # -5y^2 * x^2

witness = GF(np.array([1, out, x, y, v1, v2, v3]))

result = O_galois.dot(witness) == np.multiply(L_galois.dot(witness), R_galois.dot(witness))
assert result.all(), "result contains inequality"

## Polynomial Interpolation $O(n)$ constraints $\rightarrow$ $O(1)$
1. Turn each column into a list of polynomials for every row.
2. Given we have 4 columns, we need 4 arbitrary points to evaluate the Vector -> Polynomial homomorphism. Chose 1,2,3,4 or 0,1,2,3 or any 4 points

    ```python
    L = np.array([
        [0, 0, 1, 0,  0, 0, 0],
        [0, 0, 0, 0,  1, 0, 0],
        [0, 0, 0, 74, 0, 0, 0],
        [0, 0, 0, 0,  0, 0, 1],
    ])
    ```
- first column [0,0,0,0] should evaluate to 0
- second column should evaluate to 0
- third column [1,0,0,0] shouldn't

In [12]:
print(galois.lagrange_poly(GF([1,2,3,4]),GF([1,0,0,0])))
# 13x^3 + 41x^2 + 22x + 4

print(galois.lagrange_poly(GF([0,1,2,3]),GF([1,0,0,0])))
# 13x^3 + x^2 + 64x + 1

13x^3 + 41x^2 + 22x + 4
13x^3 + x^2 + 64x + 1


We can now interpolate all columns in the L matrix

In [14]:
# print(galois.lagrange_poly(GF([1,2,3,4]),GF([0,0,0,0])))
# print(galois.lagrange_poly(GF([1,2,3,4]),GF([0,0,0,0])))
print(galois.lagrange_poly(GF([1,2,3,4]),GF([1,0,0,0])))
print(galois.lagrange_poly(GF([1,2,3,4]),GF([0,0,74,0])))
print(galois.lagrange_poly(GF([1,2,3,4]),GF([0,1,0,0])))
# print(galois.lagrange_poly(GF([1,2,3,4]),GF([0,0,0,0])))
print(galois.lagrange_poly(GF([1,2,3,4]),GF([0,0,0,1])))

13x^3 + 41x^2 + 22x + 4
42x^3 + 22x^2 + 35x + 59
40x^3 + 75x^2 + 49x + 73
66x^3 + 78x^2 + 15x + 78


In [25]:
# done with code
def interpolate_column(col):
    xs = GF(np.array([1,2,3,4]))
    return galois.lagrange_poly(xs, col)

# axis 0 is the columns. apply_along_axis is the same as doing a for loop over the columns and collecting the results in an array
U_polys = np.apply_along_axis(interpolate_column, 0, L_galois)
V_polys = np.apply_along_axis(interpolate_column, 0, R_galois)
W_polys = np.apply_along_axis(interpolate_column, 0, O_galois)
print(U_polys)
print(V_polys)
print(W_polys)
print(witness)

[Poly(0, GF(79)) Poly(0, GF(79)) Poly(13x^3 + 41x^2 + 22x + 4, GF(79))
 Poly(42x^3 + 22x^2 + 35x + 59, GF(79))
 Poly(40x^3 + 75x^2 + 49x + 73, GF(79)) Poly(0, GF(79))
 Poly(66x^3 + 78x^2 + 15x + 78, GF(79))]
[Poly(0, GF(79)) Poly(0, GF(79)) Poly(13x^3 + 41x^2 + 22x + 4, GF(79))
 Poly(39x^3 + 43x^2 + 72x + 4, GF(79))
 Poly(27x^3 + 74x^2 + 64x + 72, GF(79)) Poly(0, GF(79)) Poly(0, GF(79))]
[Poly(0, GF(79)) Poly(66x^3 + 78x^2 + 15x + 78, GF(79)) Poly(0, GF(79))
 Poly(0, GF(79)) Poly(13x^3 + 41x^2 + 22x + 4, GF(79))
 Poly(53x^3 + 76x^2 + 34x + 74, GF(79))
 Poly(39x^3 + 43x^2 + 72x + 4, GF(79))]
[ 1 15  4 77 16 19 59]


In [26]:
from functools import reduce
def inner_product_polynomials_with_witness(polys, witness):
    mul_ = lambda x, y: x * y
    sum_ = lambda x, y: x + y
    return reduce(sum_, map(mul_, polys, witness))

term_1 = inner_product_polynomials_with_witness(U_polys, witness)
term_2 = inner_product_polynomials_with_witness(V_polys, witness)
term_3 = inner_product_polynomials_with_witness(W_polys, witness)
print(term_1)
print(term_2)
print(term_3)
print(witness)

78x^3 + 76x^2 + 28x + 59
11x^3 + 77x^2 + 20x + 54
3x^3 + 40x^2 + 20x + 32
[ 1 15  4 77 16 19 59]
